# Predictive Brain Tumor Image AI Project - Preprocessing & Modeling

To make an image-processing brain tumor predictive model to automate on scale.

# Data Imbalance

Planning to do a model where we merge 3 classes into 1 non cancer class but that causes data imbalance thus I plan to make it fix that.

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt

In [ ]:
# Define Data Transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(), # when training will 50% flip the image horizontally. Making a copy of the image and flipping it may add bias, thus why we do this controlled randomness method.
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Data path
train_dir = './ImageData1-Training'
test_dir = './ImageData2-Testing'

# Dataset setup
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms['test'])
'''
Reminder:
Pytorch dataset's ImageFolder is design to expect a parent folder, and inside sub folder with the images, and these subfolders will auto be used as the label in the training.
'''


# Add labels to dataset
class_map = {
    'glioma': 1,  # cancer
    'meningioma': 0,  # normal
    'notumor': 0,  # normal
    'pituitary': 0   # normal
}
'''
As mentioned in EDA,
I am thinking of making Glioma classified as cancer in training 
and Notumor (no tumor), Meningioma, and Pituitary as no cancer in training

Allows me to keep the separated subfolder structure for the images.
Glioma is the only cancer, while the others are just no tumor or benign tumors. 
'''

train_dataset.targets = [class_map[train_dataset.classes[label]] for label in train_dataset.targets]
test_dataset.targets = [class_map[test_dataset.classes[label]] for label in test_dataset.targets]

# Dataloader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # Shuffle for training
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)   # No shuffle for testing


# Creating the Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Example CNN from scratch
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(SimpleCNN, self).__init__()
        
        # Feature extractor (convolutional layers)
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),  # output: 16 x 112 x 112

            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),  # output: 32 x 56 x 56

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),  # output: 64 x 28 x 28
        )
        
        # Classifier (fully-connected layers)
        self.classifier = nn.Sequential(
            nn.Linear(64 * 28 * 28, 128),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(128, num_classes)  # num_classes=2 for binary classification
        )

    def forward(self, x):
        x = self.features(x)
        # Flatten the output of the convolutional layers
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Model Parameters

In [ ]:
# Choose your device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Instantiate the model
model = SimpleCNN(num_classes=2)

# Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()  # good for multi-class or binary with 'num_classes=2'
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Hyper parameters

# Training

In [ ]:
num_epochs = 10  # change as you see fit

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        # Move data to the device (GPU or CPU)
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backprop and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Accumulate loss
        running_loss += loss.item() * images.size(0)
        
        # Calculate training accuracy in this batch
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = 100.0 * correct / total
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")
